In [4]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import argparse
import os

from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

Data Normalizers

In [5]:
Train_Normalized = ImageDataGenerator(rescale = 1./255.,rotation_range = 45, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)

Test_Normalized = ImageDataGenerator(rescale = 1.0/255.)

Importing and Normalizing the Dataset

In [6]:
train_gen = Train_Normalized.flow_from_directory('./Face Mask Dataset/Train', target_size = (224,224), batch_size = 64, class_mode = 'categorical',  color_mode= 'rgb')

val_gen = Train_Normalized.flow_from_directory('./Face Mask Dataset/Validation', target_size=(224,224), class_mode='categorical', color_mode= 'rgb', batch_size= 64)

test_gen = Test_Normalized.flow_from_directory('./Face Mask Dataset/Test', target_size = (224,224), class_mode = 'categorical',color_mode= 'rgb', batch_size = 64)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [7]:
baseModel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

In [8]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)

In [9]:
model=Model(inputs=baseModel.input,outputs=headModel)

In [10]:
for layer in baseModel.layers:
    layer.trainable=False

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=2,restore_best_weights=True, verbose=1)
history = model.fit(train_gen, epochs=20, validation_data=val_gen, callbacks=[early_stop])

Epoch 1/20
157/157 [==============================] - 198s 1s/step - loss: 0.7820 - accuracy: 0.6005 - val_loss: 0.4432 - val_accuracy: 0.8000
Epoch 2/20
157/157 [==============================] - 192s 1s/step - loss: 0.4914 - accuracy: 0.7658 - val_loss: 0.2836 - val_accuracy: 0.9162
Epoch 3/20
157/157 [==============================] - 191s 1s/step - loss: 0.3632 - accuracy: 0.8413 - val_loss: 0.2285 - val_accuracy: 0.9287
Epoch 4/20
157/157 [==============================] - 193s 1s/step - loss: 0.2915 - accuracy: 0.8792 - val_loss: 0.2006 - val_accuracy: 0.9275
Epoch 5/20
157/157 [==============================] - 190s 1s/step - loss: 0.2528 - accuracy: 0.9022 - val_loss: 0.1746 - val_accuracy: 0.9450
Epoch 6/20
157/157 [==============================] - 191s 1s/step - loss: 0.2248 - accuracy: 0.9149 - val_loss: 0.1590 - val_accuracy: 0.9425
Epoch 7/20
157/157 [==============================] - 195s 1s/step - loss: 0.2059 - accuracy: 0.9219 - val_loss: 0.1472 - val_accuracy: 0.9513

In [13]:
results = model.evaluate(test_gen, verbose=0)
print("Test Loss: "+str(results[0]))
print("Test Accuracy: "+str(results[1]))

Test Loss: 0.09077750891447067
Test Accuracy: 0.9727822542190552


In [14]:
pred = model.predict(test_gen)
pred = np.argmax(pred,axis=1)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

16/16 [==============================] - 13s 719ms/step


In [15]:
model.save("Model/ask_mobileNetV2_detector.model", save_format="h5")